In [10]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from shapely.geometry import *
from shapely.ops import nearest_points
from utils import *
import warnings
import numpy as np
from sklearn.cluster import DBSCAN,AgglomerativeClustering,KMeans

In [11]:
warnings.filterwarnings("ignore")

In [12]:
crsEuc,crsDeg=getUsualCRS()
bboxEuc,bboxDeg=getUsualBbox()

waterFileDir="../../Dados/BaseLayer/water.shp"
roadsFileDir="../../Dados/BaseLayer/roads.shp"
poisFileDir="../datasets/facebookPOIS/facebookplaces.csv"

In [21]:
def getCentroid(cluster): 
    centroid = [MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y]
    return centroid

def dbscan(points,epsilonKm,minSamples,crs):
    if len(points)==0:
        raise ValueError('Empty GeoDataframe')
        
    coords = np.array([[p.x,p.y] for p in points.loc[:,('geometry')]])
    
    #dbscan  
    db = DBSCAN(eps=epsilonKm, min_samples=minSamples).fit(coords)
    
    labeledPoints =db.labels_
    uniqueClusterLabels = set(labeledPoints)

    dicti = {'geometry':[],'clusterID':[],'nPoints':[]}
    for label in uniqueClusterLabels:
        if label==-1: #outliers are marked with clusterID=-1
            continue
        else:
            cds=coords[labeledPoints==label]
            centroid=getCentroid(cds)
            dicti['geometry']=dicti.get('geometry',[])+[Point(centroid[0],centroid[1])]
            dicti['clusterID']=dicti.get('clusterID',[])+[label]
            dicti['nPoints']=dicti.get('nPoints',[])+[len(cds)]

    clusters=gpd.GeoDataFrame(data=dicti,crs=crs)

    points['clusterID']=labeledPoints.tolist()
    clustersDf=reorderDataframeIndex(clusters)
    return points,clusters

def agnes(points,nClusters,crs):
    if len(points)==0:
        raise ValueError('Empty GeoDataframe')
    
    coords = np.array([[p.x,p.y] for p in points.loc[:,('geometry')]])

    #aglomerative clustering 
    db = AgglomerativeClustering(n_clusters=nClusters).fit(coords)

    labeledPoints =db.labels_
    uniqueClusterLabels = set(labeledPoints)

    dicti = {'geometry':[],'clusterID':[],'nPoints':[]}
    for label in uniqueClusterLabels:
        if label==-1: #outliers are marked with clusterID=-1
            continue
        else:
            cds=coords[labeledPoints==label]
            centroid=getCentroid(cds)

            dicti['geometry']=dicti.get('geometry',[])+[Point(centroid[0],centroid[1])]
            dicti['clusterID']=dicti.get('clusterID',[])+[label]
            dicti['nPoints']=dicti.get('nPoints',[])+[len(cds)]
    
    clusters=gpd.GeoDataFrame(data=dicti,crs=crs)
    points['clusterID']=labeledPoints.tolist()
    clusters=reorderDataframeIndex(clusters)
    return points,clusters

def kmeans(points,nClusters,crs):
    if len(points)==0:
        raise ValueError('Empty GeoDataframe')
    
    coords = np.array([[p.x,p.y] for p in points.loc[:,('geometry')]])

    #aglomerative clustering 
    db = KMeans(n_clusters=nClusters).fit(coords)

    labeledPoints =db.labels_
    uniqueClusterLabels = set(labeledPoints)

    dicti = {'geometry':[],'clusterID':[],'nPoints':[]}
    for label in uniqueClusterLabels:
        if label==-1: #outliers are marked with clusterID=-1
            continue
        else:
            cds=coords[labeledPoints==label]
            centroid=getCentroid(cds)
            dicti['geometry']=dicti.get('geometry',[])+[Point(centroid[0],centroid[1])]
            dicti['clusterID']=dicti.get('clusterID',[])+[label]
            dicti['nPoints']=dicti.get('nPoints',[])+[len(cds)]
            
    clusters=gpd.GeoDataFrame(data=dicti,crs=crs)
    points['clusterID']=labeledPoints.tolist()
    clusters=reorderDataframeIndex(clusters)
    return points,clusters


In [22]:
minSamples=3
for i,epsilon in enumerate([300,500,700,900]):
    ficheiroComercio="../datasets/facebookPOIS/Comercio.shp"
    poisComercio=readGeodatafromFile(targetFile=ficheiroComercio,bbox=bboxEuc,crs=crsEuc)

    ficheiroComercioPois="../datasets/facebookPOIS/dbscan/points_{}_{}.shp".format(minSamples,epsilon)
    ficheiroComercioClusters="../datasets/facebookPOIS/dbscan/clusters_{}_{}.shp".format(minSamples,epsilon)
    poisComercio,clusters=dbscan(points=poisComercio,epsilonKm=epsilon,minSamples=minSamples,crs=crsEuc)

    writeGeodataToGis(geodf=poisComercio,targetFile=ficheiroComercioPois,crs=crsEuc)
    writeGeodataToGis(geodf=clusters,targetFile=ficheiroComercioClusters,crs=crsEuc)


for i,nClusters in enumerate([5,10,20]):
    ficheiroComercio="../datasets/facebookPOIS/Comercio.shp"
    poisComercio=readGeodatafromFile(targetFile=ficheiroComercio,bbox=bboxEuc,crs=crsEuc)

    ficheiroComercioPois="../datasets/facebookPOIS/agnes/points_{}.shp".format(nClusters)
    ficheiroComercioClusters="../datasets/facebookPOIS/agnes/clusters_{}.shp".format(nClusters)
    poisComercio,clusters=agnes(nClusters=nClusters,points=poisComercio,crs=crsEuc)

    writeGeodataToGis(geodf=poisComercio,targetFile=ficheiroComercioPois,crs=crsEuc)
    writeGeodataToGis(geodf=clusters,targetFile=ficheiroComercioClusters,crs=crsEuc)


for i,nClusters in enumerate([10]):
    ficheiroComercio="../datasets/facebookPOIS/Comercio.shp"
    poisComercio=readGeodatafromFile(targetFile=ficheiroComercio,bbox=bboxEuc,crs=crsEuc)

    ficheiroComercioPois="../datasets/facebookPOIS/kmeans/points_{}.shp".format(nClusters)
    ficheiroComercioClusters="../datasets/facebookPOIS/kmeans/clusters_{}.shp".format(nClusters)
    poisComercio,clusters=kmeans(nClusters=nClusters,points=poisComercio,crs=crsEuc)

    writeGeodataToGis(geodf=poisComercio,targetFile=ficheiroComercioPois,crs=crsEuc)
    writeGeodataToGis(geodf=clusters,targetFile=ficheiroComercioClusters,crs=crsEuc)

In [23]:
categories=list(taxonomy().keys())
epsilonKm=700
minSamples=3
for cat in categories:
    pointsFileDir="../datasets/facebookPOIS/{}.shp".format(cat)
    newPointsFileDir="../datasets/facebookPOIS/finalClustering/{}.shp".format(cat)
    clustersFileDir="../datasets/facebookPOIS/finalClustering/clusters{}.shp".format(cat)

    points=readGeodatafromFile(targetFile=pointsFileDir,bbox=bboxEuc,crs=crsEuc)
    newPoints,clusters=dbscan(points=points,crs=crsEuc,epsilonKm=epsilonKm,minSamples=minSamples)

    writeGeodataToGis(geodf=newPoints,crs=crsEuc,targetFile=newPointsFileDir)
    writeGeodataToGis(geodf=clusters,crs=crsEuc,targetFile=clustersFileDir)